**Open source methodology to capture types of deprived areas in Metropolitan Areas in View of the Locus Charter**

This notebook was developed as part of the CFP project funded by Notre Dame-IBM Tech Ethics Lab under the proposal 'Solving Ethical Challenges in the Design of Open-Source Environments: Scaling Urban Mapping Models in View of the Locus Charter'

Authors: Lorraine Trento Oliveira, Dr. Julio Pedrassoli, Dr. Monika Kuffer.



## Introduction

### Description

CFP funding project. 


### Prerequisite
- A Google Earth Engine account. Sign up [here](https://earthengine.google.com) if needed. 
- [Miniconda](https://docs.conda.io/en/latest/miniconda.html) or [Anaconda](https://www.anaconda.com/products/individual)


### Set up a conda environment

```
conda create -n geo python=3.8
conda activate geo
conda install geemap -c conda-forge
conda install jupyter_contrib_nbextensions -c conda-forge
jupyter contrib nbextension install --user
perhaps also: conda update -c conda-forge earthengine-api
```


## Input preparation

### Import libraries

In [6]:
#It is required to run this again everytime you start the kernel
import os
import ee
import geemap

### Upload and prepare data

The required input data depends on the local characteristics of the study area. Based on literature review, the following datasets can be used to derive the spatial attributes of the Metropolitan Area of São Paulo (MASP). 
The datasets are acquired through the GEE data catalog and by uploading datasets as assets.

In [15]:
#Create the default interactive map
Map = geemap.Map(center=(-24, -48), zoom=7)
Map

Map(center=[-24, -48], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

#### AGSN and MASP layers

**AGSN layer: deprived areas layer for the Brazilian territory, downloaded from the National Statistics Bureau at https://ibge.gov.br/geociencias/organizacao-do-territorio/tipologias-do-territorio/15788-aglomerados-subnormais.html?=&t=acesso-ao-produto**


**MASP layer: study area spatial extent layer, downloaded from the National Spatial Data Infrastructure (INDE) at https://visualizador.inde.gov.br/Ativar?url=https://ide.emplasa.sp.gov.br/geoserver/ows**


**To import assets from your EE account to the script, it is necessary to copy your own user access ID:**

In [16]:
#Import AGSN layer to GEE account:
AGSN = ee.FeatureCollection('users/lorrainetoliveira/agsn2019')
Map.addLayer(AGSN, {}, "AGSN")

#Import MASP polygon layer as ROI (region of interest):
roi = ee.FeatureCollection('users/lorrainetoliveira/masp2010')
Map.addLayer(roi, {}, "MASP")

#Clip AGSN to the ROI boundaries:
clip_agsn = AGSN.filterBounds(roi)

# Display a clipped version of AGSN layer:
Map.centerObject(ee.FeatureCollection(roi), 9)
Map.addLayer(clip_agsn, {}, 'Clip_AGSN', False)
Map

Map(bottom=18935.0, center=[-23.59841241158308, -46.53626586100811], controls=(WidgetControl(options=['positio…

#### DEM

In [17]:
#Load the DEM from GEE catalog:
nasadem = ee.Image('NASA/NASADEM_HGT/001')

#Clip DEM to the roi boundaries:
clip_dem = nasadem.clip(roi)

#Set Visualization parameters:
vizParams_dem = {
    'bands': ['elevation'],
    'min': 0,
    'max': 3300,
    'gamma': [1],
    'opacity': 1
}

#Display the DEM' clipped version: 
Map.centerObject(ee.FeatureCollection(roi), 9)
Map.addLayer(clip_dem, vizParams_dem, 'Clip_DEM')
Map

Map(bottom=74721.0, center=[-23.59841241158308, -46.53626586100811], controls=(WidgetControl(options=['positio…

#### Landsat 8

In [18]:
#Load the LANDSAT imagery from GEE catalog 
#Collection 1 is ceased from december 2021 on. This is the substitute:
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")

#Filter by date, bounds and cloud coverage:
l8_filtered = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA") \
  .filterDate('2019-01-01', '2019-12-31') \
  .filterBounds(roi) \
  .filterMetadata('CLOUD_COVER', 'less_than', 1)

#Select only the panchromatic band: 
band_pan = l8_filtered.select(['B8'])

#Unmask to ensure we have the same number of values everywhere:
band_pan = band_pan.map(lambda i: i.unmask(-1))

#Check number of suitable images: 
band_pan.aggregate_array('CLOUD_COVER').getInfo()

#Mosaicking: spatially assemble the images to produce a spatially continuous image:
l8_mosaic = band_pan.mosaic()

#Clip to ROI:
l8_prep = l8_mosaic.clip(roi)

#Define visualization parameters:
vizParams_b8 = {'bands': ['B8'],
             'min': 0.04, 'max': 0.16,'gamma': 1}

#Center the map on the image and display:
Map.centerObject(ee.FeatureCollection(roi), 9)
Map.addLayer(l8_prep, vizParams_b8, 'L8_PAN')
Map

Map(bottom=74682.0, center=[-23.59841241158308, -46.53626586100811], controls=(WidgetControl(options=['positio…

#### Sentinel 2A

In [19]:
#Load the Sentinel 2A imagery from GEE catalog: 
s2 = ee.ImageCollection("COPERNICUS/S2_SR")

#Filter by date, bounds and cloud coverage:
s2_filtered = ee.ImageCollection("COPERNICUS/S2_SR") \
  .filterDate('2019-01-01', '2019-12-31') \
  .filterBounds(roi) \
  .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 0.15) 

#Select only  band 8(NIR) and band 4(red): 
s2_b8 = s2_filtered.select(['B8'])
s2_b4 = s2_filtered.select(['B4'])

#Check number of suitable images: 
s2_b8.aggregate_array('CLOUDY_PIXEL_PERCENTAGE').getInfo() #Check also the map if they cover the entire ROI 

#Unmask to ensure we have the same number of values everywhere:
s2_b8 = s2_b8.map(lambda i: i.unmask(-1))
s2_b4 = s2_b4.map(lambda i: i.unmask(-1))

#Reduces an image collection by calculating the median of all values at each pixel across the stack of all matching bands:
b8_median = s2_b8.reduce(ee.Reducer.median()) \
  .clip(roi)
b4_median = s2_b4.reduce(ee.Reducer.median()) \
  .clip(roi)

#Define visualization parameters:
vizParams_s2 = {'min': 100, 'max': 5000,'gamma': 1}

# Center the map on the image and display:
Map.centerObject(ee.FeatureCollection(roi), 9)
Map.addLayer(b8_median, vizParams_s2, 'b8')
Map.addLayer(b4_median, vizParams_s2, 'b4')
Map

Map(bottom=74682.0, center=[-23.59841241158308, -46.53626586100811], controls=(WidgetControl(options=['positio…

#### World Settlement Footprint 2019

**First, download the tiles from https://download.geoservice.dlr.de/WSF2019/files/ considering the respective roi coordinates. Then, upload them to the EE assets as GeoTIFF image files:**

In [20]:
#Load the images:
WSF_4624 = ee.Image('users/lorrainetoliveira/WSF2019_v1_-46_-24')
WSF_4824 = ee.Image('users/lorrainetoliveira/WSF2019_v1_-48_-24')

#Mosaic the two images to produce a spatially continuous image:
WSF_mosaic = ee.ImageCollection([WSF_4624,WSF_4824])

#Unmask to ensure we have the same number of values everywhere:
WSF_mosaic = WSF_mosaic.map(lambda i: i.unmask(-1))

#Reduces an image collection to an image by calculating the median of all values at each pixel across the stack:
WSF_median = WSF_mosaic.reduce(ee.Reducer.median()) \
  .clip(roi)

#Define visualization parameters:
vizParams_wsf = {'min': 0, 'max': 255,'gamma': 1}

#Center the map on the image and display:
Map.centerObject(ee.FeatureCollection(roi), 9)
Map.addLayer(WSF_median, vizParams_wsf, 'WSF')
Map

Map(bottom=74682.0, center=[-23.59841241158308, -46.53626586100811], controls=(WidgetControl(options=['positio…

#### WorldPop

In [21]:
#Load the layer from GEE catalog:
GPop = ee.ImageCollection('WorldPop/GP/100m/pop')

#Select only the panchromatic band: 
GPop_pop = GPop.select(['population'])

#Mosaicking: spatially assemble the images to produce a spatially continuous image:
GPop_mosaic = GPop_pop.mosaic()

#Clip to ROI:
Gpop_clip = GPop_mosaic.clip(roi)

# -----------------------
MaxValues = GPop_mosaic.reduceRegion(**{
    'reducer': ee.Reducer.max(),
    'geometry': roi,
    'scale': 100,
    'crs': 'EPSG:4326',
})
                                
print(MaxValues.getInfo())

{'population': 3691.753173828125}


In [22]:
#Set Visualization parameters:
vizParams_gpop = {'min': 0.0, 'max': 50.0,'palette': ['24126c', '1fff4f', 'd4ff50']}

#Center the map on the image and display:
Map.centerObject(ee.FeatureCollection(roi), 9)
Map.addLayer(Gpop_clip, vizParams_gpop, 'WorldPop')
Map

Map(bottom=74682.0, center=[-23.59841241158308, -46.53626586100811], controls=(WidgetControl(options=['positio…

#### OSM layers

 - First, download OSM layers from https://export.hotosm.org/ and upload as .shp (or .zip) asset to your GEE account. Perhaps more than one export need to be done in order to cover the entire ROI. (Remember to separate the layers per geometry feature before uploading to GEE)
 - Then, some data cleanup is necessary because osm includes way more information than we need (considering the variables chosen in the project)

In [ ]:
# packages
! pip install geopandas
! pip install cartopy
! pip install osmnx

In [31]:
#Import vector line layers to GEE account:
SP_lines = ee.FeatureCollection('users/lorrainetoliveira/RMSP_planet_osm_line_lines')
SP_lines2 = ee.FeatureCollection('users/lorrainetoliveira/RMSP2_lines')

osm_l = SP_lines.merge(SP_lines2)

#Clip AGSN to the ROI boundaries:
osm_lines = osm_l.filterBounds(roi)

#Layer preparation using Graphical Modeler in QGIS
#Please access the following 



# Display a clipped version of AGSN layer:
Map.centerObject(ee.FeatureCollection(roi), 9)
Map.addLayer(osm_lines, {}, 'OSM Line Features', False)
Map

Map(center=[-23.59841241158308, -46.53626586100811], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
#trials
Map = geemap.Map()

#OSM = geemap.osm_to_ee("Sao Paulo, Brasil")
#geemap.osm_to_geopandas
#geemap.osm_to_geojson


Map.add_osm("Sao Paulo, Sao Paulo, Brasil", layer_name="Sao Paulo, Brazil", to_ee=True)

Map.addLayer(OSM, {}, "Sao Paulo")
Map.centerObject(OSM, 9)
Map